Create Player Ratings

In [1]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [48]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/player_stats.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

finals_round_map = {'F1':24,
                    'F2':25,
                    'F3':26,
                    'F4':27}
player_stats['Round_ID'] = player_stats['Round_ID'].astype(str)
player_stats['Round'] = (player_stats['Round_ID'].str[4:].replace(finals_round_map)).astype(int)
player_stats['Round_str'] = player_stats['Round_ID'].str[4:].replace(finals_round_map)
player_stats['Round_ID_num'] = (player_stats['Season'].astype(str) + player_stats['Round'].astype(str)).astype(int)

player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig 

In [49]:
player_stats[['Position_Group', 'Position_Sub_Group', 'Position']].value_counts().sort_index()

Position_Group  Position_Sub_Group  Position                
Back            Back-Pocket         Back Pocket Left            1130
                                    Back Pocket Right           1119
                Half-Back           Half Back Flank Left        1118
                                    Half Back Flank Right       1113
                Key-Back            Centre Half Back            1113
                                    Full Back                   1133
Forward         Forward-Pocket      Forward Pocket Left         1112
                                    Forward Pocket Right        1121
                Half-Forward        Half Forward Flank Left     1109
                                    Half Forward Flank Right    1099
                Key-Forward         Centre Half Forward         1104
                                    Full Forward                1143
Interchange     Interchange         Interchange                 4466
                                    Substi

Different Positions are expected to do different things

In [50]:
match = player_stats[player_stats['Match_ID'] == "202101_BrisbaneLions_Sydney"]
match.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig 

Team Ratings - Combination of VAEP and Expected VAEP Values should be a correct order (Season by Season)

In [51]:
player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84,0,1.0,0.0,0,1,1.0,100.0,1.0,1,55.6,5,0.0,0.0,0.0,4.0,88.9,9,31,8.0,5.0,0.0,0,1,0.0,0,0,3.0,3,0,0.0,0.0,0.0,0,1.0,4,83.3,0.0,0.0,6,2.0,2,0,1.0,172.0,1,5.0,6.1,2,0.0,4,1.0,0,1.0,1,0,0,1,9,2,4,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87,0,0.0,0.0,0,4,1.0,100.0,1.0,0,66.7,8,0.0,0.0,0.0,5.0,83.3,12,36,10.0,5.0,0.0,2,0,0.0,1,0,8.0,6,0,0.0,0.0,0.0,2,0.0,7,83.3,1.0,1.0,6,1.0,0,0,0.0,258.0,2,7.0,8.0,1,0.0,2,1.0,0,2.0,0,3,1,0,12,6,4,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77,0,0.0,9.0,3,0,0.0,0.0,0.0,0,50.0,5,0.0,0.0,0.0,3.0,70.0,10,39,7.0,4.0,0.0,0,2,0.0,1,0,1.0,3,0,0.0,0.0,0.0,3,0.0,1,57.1,0.0,0.0,7,2.3,2,1,2.0,207.0,1,9.0,10.1,0,0.0,4,1.0,0,0.0,0,1,0,3,10,0,5,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0,0.000000,Brisbane Lions,Sydney,Sydney,1,01,20211
3,202101_BrisbaneLions_Sydney,Brisbane Lions,Daniel Rich,202101,Daniel_Rich,MEDIUM_DEFENDER,CD_I271072,31,183,95,10,LEFT,WA,2008.0,2009.0,Sorrento-Duncraig 

In [52]:
sorted(list(set(player_stats['Round_ID'])))[-10:]

['202312',
 '202313',
 '202314',
 '202315',
 '202316',
 '202317',
 '202318',
 '202319',
 '202320',
 '202321']

In [66]:
def create_team_summary(player_stats, rounds = None):
    
    player_stats_copy = player_stats.copy()
    
    if rounds is not None:
        all_rounds = sorted(list(set(player_stats_copy['Round_ID'])))
        rating_rounds = all_rounds[-rounds:]
        player_stats_copy = player_stats_copy[player_stats_copy['Round_ID'].isin(rating_rounds)]
    
    team_for = player_stats_copy.groupby('Team').sum()[['Score', 'xScore','vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_for.columns = [x+"_F" for x in list(team_for)]
    team_against = player_stats_copy.groupby(['Opponent']).sum()[['Score', 'xScore', 'vaep_value', 'vaep_value_received', 'offensive_value', 'defensive_value', 'exp_vaep_value', 'exp_vaep_value_received', 'exp_offensive_value', 'exp_defensive_value']]
    team_against.columns = [x+"_A" for x in list(team_against)]
    team_games = player_stats_copy.groupby(['Match_ID', 'Team']).size().reset_index().groupby('Team').count()[['Match_ID']]
    team_games.columns = ['Games']
    team_summary = pd.concat([team_games, team_for, team_against], axis=1)
    
    for col in [x.replace("_F", "") for x in list(team_for)]:
        team_summary[col+"_pergame_F"] = team_summary[col+"_F"] / team_summary["Games"]
        team_summary[col+"_pergame_A"] = team_summary[col+"_A"] / team_summary["Games"]

        team_summary[col+"_diff"] = team_summary[col+"_F"] - team_summary[col+"_A"]
        team_summary[col+"_pergame_diff"] = team_summary[col+"_pergame_F"] - team_summary[col+"_pergame_A"]

    return team_summary


In [68]:
team_summary = create_team_summary(player_stats, rounds = None)
team_summary

,Games,Score_F,xScore_F,vaep_value_F,vaep_value_received_F,offensive_value_F,defensive_value_F,exp_vaep_value_F,exp_vaep_value_received_F,exp_offensive_value_F,exp_defensive_value_F,Score_A,xScore_A,vaep_value_A,vaep_value_received_A,offensive_value_A,defensive_value_A,exp_vaep_value_A,exp_vaep_value_received_A,exp_offensive_value_A,exp_defensive_value_A,Score_pergame_F,Score_pergame_A,Score_diff,Score_pergame_diff,xScore_pergame_F,xScore_pergame_A,xScore_diff,xScore_pergame_diff,vaep_value_pergame_F,vaep_value_pergame_A,vaep_value_diff,vaep_value_pergame_diff,vaep_value_received_pergame_F,vaep_value_received_pergame_A,vaep_value_received_diff,vaep_value_received_pergame_diff,offensive_value_pergame_F,offensive_value_pergame_A,offensive_value_diff,offensive_value_pergame_diff,defensive_value_pergame_F,defensive_value_pergame_A,defensive_value_diff,defensive_value_pergame_diff,exp_vaep_value_pergame_F,exp_vaep_value_pergame_A,exp_vaep_value_diff,exp_vaep_value_pergame_diff,exp_vaep_value_received_pergame_F,exp_vaep_value_received_pergame_A,exp_vaep_value_received_diff,exp_vaep_value_received_pergame_diff,exp_offensive_value_pergame_F,exp_offensive_value_pergame_A,exp_offensive_value_diff,exp_offensive_value_pergame_diff,exp_defensive_value_pergame_F,exp_defensive_value_pergame_A,exp_defensive_value_diff,exp_defensive_value_pergame_diff
Adelaide,62,4929,5028.934018,1084.057041,981.493062,907.454954,176.602087,6960.235953,6369.528286,6170.954294,789.281659,5275,5284.098872,1133.792414,1021.261302,960.999456,172.792958,7190.550272,6806.493562,6488.422212,702.128059,79.500000,85.080645,-346,-5.580645,81.111839,85.227401,-255.164854,-4.115562,17.484791,18.286974,-49.735373,-0.802183,15.830533,16.471956,-39.768240,-0.641423,14.636370,15.499991,-53.544502,-0.863621,2.848421,2.786983,3.809129,0.061438,112.261870,115.976617,-230.314319,-3.714747,102.734327,109.782154,-436.965276,-7.047827,99.531521,104.651971,-317.467918,-5.120450,12.730349,11.324646,87.153600,1.405703
Brisbane Lions,67,6106,6212.882315,1294.764093,1104.590580,1110.345577,184.418515,8181.518445,7267.885030,7371.394882,810.123555,5052,5159.341999,1145.775168,1067.578157,971.576741,174.198427,7206.454739,7043.906890,6412.410969,794.043771,91.134328,75.402985,1054,15.731343,92.729587,77.005104,1053.540316,15.724482,19.324837,17.101122,148.988925,2.223715,16.486427,15.934002,37.012423,0.552424,16.572322,14.501145,138.768836,2.071177,2.752515,2.599977,10.220089,0.152539,122.112216,107.559026,975.063706,14.553190,108.475896,105.132939,223.978140,3.342957,110.020819,95.707626,958.983913,14.313193,12.091396,11.851400,16.079783,0.239997
Carlton,63,5077,5471.901580,1110.021087,1027.470557,935.316587,174.704501,7469.877309,7017.066691,6623.738903,846.138410,4915,5084.279246,1092.376498,1018.493251,920.935125,171.441373,7126.263086,6611.317827,6299.376034,826.887050,80.587302,78.015873,162,2.571429,86.855581,80.702845,387.622334,6.152735,17.619382,17.339309,17.644589,0.280073,16.309056,16.166560,8.977306,0.142497,14.846295,14.618018,14.381462,0.228277,2.773087,2.721292,3.263127,0.051796,118.569481,113.115287,343.614223,5.454194,111.382011,104.941553,405.748864,6.440458,105.138713,99.990096,324.362869,5.148617,13.430768,13.125191,19.251360,0.305577
Collingwood,65,5154,5317.128131,1118.554075,1059.458411,943.570109,174.983966,7260.509650,6971.994165,6436.416673,824.092972,4916,5101.377116,1092.791530,993.902260,930.837677,161.953853,7062.258647,6704.285541,6272.310943,789.947700,79.292308,75.630769,238,3.661538,81.801971,78.482725,215.751016,3.319246,17.208524,16.812177,25.762545,0.396347,16.299360,15.290804,65.556151,1.008556,14.516463,14.320580,12.732433,0.195884,2.692061,2.491598,13.030112,0.200463,111.700148,108.650133,198.251003,3.050015,107.261449,103.142854,267.708624,4.118594,99.021795,96.497091,164.105730,2.524704,12.678353,12.153042,34.145272,0.525312
Essendon,64,5192,5321.666205,1122.405649,1037.829628,961.649622,160.756027,7245.409756,6824.512535,6479.128983,766.280780,54

In [70]:
def zscale_stats(summary_data, stats):
    
    for col in stats:
        summary_data[col+'_z'] = (summary_data[col] - summary_data[col].mean()) / np.std(summary_data[col])
    summary_data = summary_data.fillna(0)
    
    return summary_data

In [74]:
rating_factors = ['Score_pergame_diff', 'xScore_pergame_diff', 'vaep_value_pergame_diff', 'exp_vaep_value_pergame_diff']
team_ratings = team_summary[rating_factors]
team_ratings = zscale_stats(team_ratings, rating_factors)
team_ratings['team_rating'] = team_ratings[[x+"_z" for x in rating_factors]].mean(axis=1)
team_ratings.sort_values(by = 'team_rating', ascending = False)

,Score_pergame_diff,xScore_pergame_diff,vaep_value_pergame_diff,exp_vaep_value_pergame_diff,Score_pergame_diff_z,xScore_pergame_diff_z,vaep_value_pergame_diff_z,exp_vaep_value_pergame_diff_z,team_rating
Melbourne,19.573529,20.316081,2.839715,18.906802,1.292712,1.369364,1.316334,1.325347,1.325939
Geelong,20.558824,17.723470,2.944096,17.323496,1.356503,1.197814,1.363779,1.216458,1.283638
Brisbane Lions,15.731343,15.724482,2.223715,14.553190,1.043956,1.065544,1.036339,1.025934,1.042943
Western Bulldogs,11.202899,12.571194,1.543866,12.661062,0.750769,0.856895,0.727323,0.895806,0.807698
Port Adelaide,11.092308,11.003225,1.613596,11.232388,0.743609,0.753144,0.759018,0.797552,0.763331
Sydney,12.848485,8.674534,1.912459,8.237212,0.857310,0.599058,0.894862,0.591563,0.735698
Richmond,4.906250,4.155359,0.804716,4.159975,0.343103,0.300030,0.391352,0.311158,0.336411
Carlton,2.571429,6.152735,0.280073,5.454194,0.191939,0.432194,0.152882,0.400166,0.294295
Collingwood,3.661538,3.319246,0.396347,3.050015,0.262516,0.244705,0.205733,0.234822,0.236944
St Kilda,-0.838710,-2.026772,-0.351096,-1.387783,-0.028845,-0.109034,-0.134007,-0.070380,-0.085566


In [75]:
def create_team_ratings(team_summary, rating_factors = []):
    
    team_ratings = team_summary[rating_factors]
    team_ratings = zscale_stats(team_ratings, rating_factors)
    team_ratings['team_rating'] = team_ratings[[x+"_z" for x in rating_factors]].mean(axis=1)
    
    return team_ratings.sort_values(by = 'team_rating', ascending = False)

In [56]:
def get_cumulative_round_summary(player_stats, current_round_id, rating_factor, num_rounds = None):
    
    current_season = int(current_round_id[:4])
    if "F" in current_round_id:
        current_round_id_int = int(finals_round_map[current_round_id[4:]])
        current_round_id_str = finals_round_map[current_round_id[4:]]
    else:
        current_round_id_int = int(current_round_id[4:])
        current_round_id_str = current_round_id[4:]
    current_round_id_num = int(str(current_season) + str(current_round_id_str))

    past_seasons_player_stats = player_stats[(player_stats['Season'] < current_season)]
    current_round_player_stats = player_stats[(player_stats['Season'] == current_season) & (player_stats['Round'] <= current_round_id_int)]
    rating_player_stats = pd.concat([past_seasons_player_stats, current_round_player_stats], axis=0)

    round_summary_dict = {}
    for round_id in list(set(rating_player_stats['Round_ID'])):
        season = int(round_id[:4])
        if "F" in round_id:
            round_id_int = int(finals_round_map[round_id[4:]])
            round_id_str = finals_round_map[round_id[4:]]
        else:
            round_id_int = int(round_id[4:])
            round_id_str = round_id[4:]
        seasons_player_stats = rating_player_stats[(rating_player_stats['Season'] < season)]
        round_player_stats = rating_player_stats[(rating_player_stats['Season'] == season) & (rating_player_stats['Round'] <= round_id_int)]
        latest_round_player_stats = pd.concat([seasons_player_stats, round_player_stats], axis=0)
        
        latest_round_team_summary = create_team_summary(latest_round_player_stats, rounds = num_rounds)
        
        round_summary_dict[int(str(season) + str(round_id_str))] = latest_round_team_summary[rating_factor]
        
    round_summary_df = pd.DataFrame.from_dict(round_summary_dict)
    # round_summary_df.T.sort_index()
    round_summary_df = round_summary_df.reindex(sorted(round_summary_df.columns), axis=1).sort_values(by=int(current_round_id_num), ascending=False)
    return round_summary_df

In [59]:
get_cumulative_round_summary(player_stats, current_round_id='202320', rating_factor = 'xScore_pergame_diff', num_rounds = 10)


,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320
Brisbane Lions,-23.111579,-11.450575,-4.207421,-11.914783,1.784022,2.786343,7.911399,10.476015,16.618264,18.021818,25.428385,22.202228,23.527044,32.008533,30.452157,31.502476,25.741411,22.403460,21.372943,18.261218,19.842212,29.020036,28.597565,21.866150,18.714011,18.876589,22.884643,25.938976,18.419086,30.569832,19.470628,15.434593,14.138207,19.020174,24.786036,23.416785,20.537085,22.160021,22.178155,15.278596,19.599618,11.396484,12.174329,8.270416,6.050947,4.992197,5.974989,4.230813,6.351347,0.090408,1.044756,5.242708,-0.550436,1.843637,-11.642619,-9.653354,-10.271066,-10.005858,-13.205169,-6.849903,-2.682279,1.685207,11.838302,15.181476,24.701967,25.185036,24.635521,26.568819,26.122915,31.319489,36.676208,35.725226,31.366848,23.848889
Carlton,-38.394501,-25.117241,2.886495,13.781812,10.994837,7.862707,9.208013,5.300529,2.567974,2.893670,5.597762,7.369544,1.644608,-3.622998,-1.580474,-1.319781,-3.492989,3.663875,4.831674,3.403044,3.239107,-5.236894,-8.500446,-9.340418,-12.786678,-13.834530,-15.760448,-18.321062,-18.981257,-16.958804,-18.714955,-4.702129,-3.591456,7.033829,12.358389,15.170732,16.411326,10.445523,13.022073,13.513134,15.420754,15.646730,21.745684,21.025719,11.805106,12.495142,6.744888,7.779176,6.798942,7.789345,9.343421,8.548276,6.386286,-3.266976,2.957467,-1.179384,6.145836,11.723529,3.648595,0.499255,13.955890,9.542379,8.349198,5.207894,4.508649,-1.497120,-3.569571,0.666825,6.349001,11.685679,5.313862,10.366976,18.308681,19.802274
Greater Western Sydney,13.394342,-11.175611,-19.131545,-12.832163,-7.831079,-13.799936,-5.253563,-3.000749,-2.037923,1.164232,-5.270611,-1.884505,3.268642,2.699165,-0.976964,3.635471,-1.034850,-2.859040,-2.168883,-7.295727,-2.181157,0.623461,3.277211,0.254373,-0.724167,-0.567707,-1.137644,-1.415151,-5.903230,-1.283885,-5.735111,-13.899742,-20.183757,-12.239648,-13.159845,-15.883137,-9.435623,-12.753819,-11.501073,-10.466077,-9.752068,-8.999125,-5.409580,-12.127538,-12.032345,-12.722381,-26.530462,-18.417039,-17.811378,-23.792411,-22.646379,-21.392734,-27.292581,-27.475273,-19.867551,-13.070969,-3.753464,-13.201329,-13.369509,-12.500265,-13.494392,-11.060812,-13.988336,-15.898645,-15.249184,-16.660422,-9.844332,-2.015582,-2.015582,-3.296067,0.839741,2.114891,12.723186,16.763196
Port Adelaide,33.394971,29.080731,11.946732,15.824953,12.690575,15.537522,7.794770,13.879335,10.738450,8.822273,7.515289,5.598488,6.999972,9.390975,10.956429,10.806833,10.496274,5.763600,10.281238,13.012493,10.994746,16.935889,19.935155,18.495813,18.968442,11.029941,17.590780,13.003449,7.958651,7.113916,2.869356,-7.640264,-3.488290,-8.836185,-4.868838,6.937228,2.764183,5.491353,7.666867,2.033686,7.249561,8.321390,0.664027,4.485623,1.395506,-5.715003,-2.450010,-7.808687,-1.268764,8.693262,7.975645,9.475642,13.031552,10.837406,25.023285,18.300561,15.794416,21.282319,15.546483,12.265247,10.854162,15.083440,19.280769,19.769658,14.320263,21.793475,26.506700,30.955095,31.819492,25.644926,29.649276,22.003251,19.355599,16.670081
Melbourne,29.261231,36.044227,35.710622,34.113076,39.771494,34.333410,32.774182,28.424137,27.409506,25.800446,25.207164,24.129556,19.862275,18.811368,13.338753,12.782011,14.786228,17.246667,15.514007,24.035037,24.805074,24.994304,26.195000,26.177262,27.624554,35.162023,36.646233,39.705878,38.651414,33.434529,33.604787,34.237877,40.603346,38.736098,37.483721,36.932137,36.047435,29.083273,27.911714,20.425707,19.162890,20.953466,18.702584,12.409524,9.601612,2.215815,1.227348,5.114861,7.2

In [63]:
get_cumulative_round_summary(player_stats, current_round_id='202320', rating_factor = 'vaep_value_pergame_diff', num_rounds = 10)

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320
Collingwood,-1.976946,0.323010,-0.036267,-0.895375,-1.634360,-1.855695,-2.247037,-1.727248,-2.075646,-1.873471,-1.879196,-1.936879,-1.680585,-1.481461,-1.206204,-1.100526,-0.379309,-1.058453,-0.949256,-0.094699,-0.186842,-2.046402,-3.024807,-3.024807,-3.138785,-3.299858,-4.165812,-2.990145,-1.236537,-3.212327,-3.349391,-1.053397,0.410716,0.585744,-0.072427,-0.910232,-0.333296,-0.515628,-1.192091,-0.363565,0.005579,-0.017186,-0.018627,-0.108197,0.298288,1.241240,0.722878,0.826983,0.415919,0.045233,-0.090324,0.094900,0.034754,-0.053582,0.376246,1.618320,1.714654,0.783373,1.359123,1.583579,1.742842,2.358944,3.383324,3.640637,3.954493,3.694243,3.669811,4.930401,4.495980,5.458453,5.784266,5.882273,4.936281,3.832281
Carlton,-3.841816,-3.232392,0.057321,0.269658,-0.933908,-1.288033,-0.545511,-0.947161,-1.196945,-0.699937,-0.692552,-0.720645,-1.538132,-2.402674,-1.555387,-0.784660,-1.726497,-0.842082,-1.235313,-1.003560,-0.924187,-2.285334,-2.160200,-1.636421,-2.075652,-2.926156,-2.652698,-2.877345,-1.267217,-2.249883,-2.521149,0.019270,-0.609430,0.391676,1.181333,1.473151,1.497353,1.109073,0.907139,1.556867,2.000090,2.527195,2.737872,2.897374,1.722984,1.703210,0.995776,0.671705,0.540792,-0.113400,-0.049089,-0.659256,-0.337708,-1.699630,-0.699999,-1.119131,-0.352886,1.014284,-0.343276,-0.686778,1.767186,1.076041,0.622389,-0.009764,-0.582014,-0.860527,-1.607246,-1.175680,-0.330446,1.047398,0.047697,1.226953,2.717683,3.821683
Sydney,5.024261,5.940071,6.158444,4.412907,3.528253,1.980075,2.107704,1.640980,1.998963,1.883404,1.757774,1.160537,-0.183042,-0.111857,-0.436917,1.803683,1.710122,2.096453,2.207904,2.250705,1.366985,1.471990,3.835205,3.338859,4.036054,2.739987,2.841210,3.290122,3.579598,3.426575,4.201888,5.053535,3.833517,3.669033,2.950320,3.451409,2.934751,2.425396,1.729747,1.904451,1.317481,1.325237,0.315035,1.489709,2.049055,1.750213,3.247522,3.840689,4.161455,3.945613,4.730749,4.383068,4.608722,2.427885,2.845404,3.662194,1.361116,0.807162,0.430229,-1.650060,-2.112666,-2.507061,-2.829327,-1.536451,-1.883765,-3.634442,-2.731076,-2.873412,0.198978,1.722355,1.680579,2.423625,3.238773,3.230587
Greater Western Sydney,-2.239149,-3.148871,-3.446974,-1.717055,-1.371572,-2.107050,-0.290793,-0.248043,-0.031292,0.248167,-0.288865,0.129994,0.599430,0.977375,0.740511,1.618189,0.242580,0.031235,0.076801,-0.668991,0.725663,1.111928,1.197153,0.622558,0.372294,0.178697,0.422751,0.029954,-0.837620,0.141098,-1.081571,-3.089810,-3.441672,-2.461841,-2.729972,-2.849720,-1.819164,-1.430382,-1.052893,-0.618493,-0.316046,1.083556,1.799803,-0.160175,-0.035224,-0.015451,-2.150613,-1.725544,-1.646584,-2.795613,-2.871024,-3.369756,-4.517623,-3.825078,-2.422089,-2.477825,-0.508675,-1.003666,-1.017211,-0.650311,-0.560170,-0.890887,-1.950497,-1.844531,-1.958527,-1.741650,-1.418356,-0.204898,-0.204898,0.213796,0.241655,0.839086,2.629408,2.741105
Adelaide,NaN,-6.855882,-2.368974,0.432465,0.308624,0.423069,-1.415233,-2.159330,-2.466407,-2.074836,-2.309682,-1.828707,-1.829269,-2.703114,-2.904728,-3.973829,-3.741427,-3.631848,-2.791387,-3.654015,-3.155090,-3.655366,-3.362698,-3.362698,-3.548358,-2.806543,-1.855112,-0.568420,-2.166718,-0.769736,-0.635128,1.124535,0.353534,-0.977958,-1.694329,-2.217572,-2.566669,-3.554215,-2.404302,-2.857666,-3.814625,-3.310508,-3.640092,-3.076797,-2.217887,-2.036445,-0.885303,0.265885,0.201228,-0.593421,0.200415,-0.842834,-0.556676,0.000085,-0.632354,-0.453177,-0.450903,0.104442,0.855634,2.077880,1.786216,1.321857,2.338668,

In [65]:
get_cumulative_round_summary(player_stats, current_round_id='202320', rating_factor = 'exp_vaep_value_pergame_diff', num_rounds=10)

,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202113,202114,202115,202116,202117,202118,202119,202120,202121,202122,202123,202124,202125,202126,202127,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202213,202214,202215,202216,202217,202218,202219,202220,202221,202222,202223,202224,202225,202226,202227,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202313,202314,202315,202316,202317,202318,202319,202320
Brisbane Lions,-20.508690,-10.518305,-3.121426,-10.698326,2.246935,3.487617,8.086144,10.429470,16.130000,17.657124,24.382186,21.484922,22.575210,31.038360,29.106689,29.870971,24.221925,21.078263,20.305905,16.939385,18.308475,27.143805,26.656098,19.942874,17.301982,17.383363,21.451797,24.297899,17.454048,29.146206,18.766349,14.734917,13.614332,17.944519,23.411644,21.546420,18.249025,19.913932,19.407545,12.974088,17.079556,9.202503,10.026538,6.701860,5.062971,4.662053,6.280852,4.550008,6.961133,1.000190,1.945978,5.449048,-0.277131,1.878023,-12.216485,-10.493026,-11.294172,-11.618201,-15.154036,-9.386606,-5.231966,-0.295705,10.197156,13.639872,23.936908,24.613085,23.824848,26.334311,25.499665,30.067236,34.734881,33.475403,29.044835,21.727676
Carlton,-37.800390,-25.606683,1.174708,12.632183,10.620424,7.235182,8.391625,4.573764,1.767470,2.256994,5.299252,7.064046,1.766997,-3.807684,-1.937663,-1.351894,-3.187549,3.616098,5.235327,3.837945,3.329216,-4.878879,-8.204208,-8.763840,-12.284767,-13.408433,-15.444736,-17.684425,-19.524551,-17.332168,-19.685963,-6.331162,-5.366540,4.820026,9.864239,13.113335,14.009120,8.186960,11.002242,11.426202,13.664920,14.174614,19.699846,19.229854,10.499835,10.226994,5.297664,6.430785,5.694039,6.611601,8.005188,7.121066,5.655126,-3.687269,1.306651,-1.340408,6.011589,10.347942,2.197489,-1.435753,11.825437,7.673099,6.748553,3.947270,4.202981,-1.751503,-3.525462,1.096128,6.755630,12.224433,5.915579,11.231220,18.870884,20.230139
Greater Western Sydney,15.619497,-11.459450,-18.874238,-13.754749,-7.685667,-13.814229,-5.975894,-3.449112,-2.243230,1.056543,-5.179600,-1.473067,3.526398,3.700155,-0.411178,4.350648,0.609872,-0.439227,-0.041337,-5.007145,-0.093306,2.668116,5.352196,1.792829,0.567551,0.808457,-0.013877,-1.040192,-5.507450,-0.854559,-5.311262,-13.366539,-20.048212,-11.509797,-12.097589,-15.098534,-9.155257,-12.609620,-11.260331,-10.623473,-10.586412,-10.445578,-7.257440,-13.708732,-13.914797,-13.641956,-26.412502,-18.717915,-18.108479,-24.030260,-22.496127,-20.851418,-25.740484,-26.333822,-18.728958,-12.803015,-4.672769,-14.418303,-14.777252,-13.296345,-13.716579,-11.756313,-13.682564,-15.917874,-14.985449,-16.517486,-9.268662,-1.074740,-1.074740,-2.685251,1.662392,3.040368,12.441657,16.391129
Melbourne,29.955299,35.765019,35.077950,33.136901,38.119502,32.228493,30.142569,25.421782,24.895238,23.439803,22.879391,21.671973,17.552197,16.467579,11.452801,11.322484,14.232263,17.362074,15.187607,23.807090,24.400424,24.615818,25.716012,25.583702,26.991124,34.215370,35.010212,37.767263,37.369289,31.871244,31.833070,32.442201,38.707088,36.653178,35.205733,34.470062,33.723555,27.245947,25.964500,18.359642,17.138757,18.939449,16.844198,10.766400,7.997436,1.022338,-0.101494,3.990359,5.792607,12.565717,10.588030,7.084959,3.435008,9.963441,14.993744,13.117030,10.764199,13.612580,9.471823,6.339404,13.558741,11.232549,14.396254,10.601965,8.628846,14.359263,15.649525,12.922176,15.645314,19.979416,13.493848,12.433265,8.420718,15.174033
Port Adelaide,29.158741,26.456084,9.831398,14.749026,11.284544,14.254748,7.121597,13.211351,10.093772,8.416047,7.428139,5.614218,7.463084,9.803178,11.757522,11.239904,10.335712,5.879254,10.566270,12.857521,11.384028,17.209414,19.886419,18.313086,18.679461,11.441242,18.348277,13.718263,8.961010,8.289340,3.868610,-6.147491,-2.237182,-7.667119,-3.892279,7.003245,3.255571,5.690487,7.514067,2.160269,7.376091,8.096259,0.841819,4.262989,1.644712,-4.367762,-1.338299,-5.789665,-0.008461